<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame(data={
    'x1': [0,0,1,0,1,1,0],
    'x2': [0,1,0,1,0,1,0],
    'x3': [1,1,1,0,0,1,0],
    'y': [0,1,1,1,1,0,0]
})

df

,x1,x2,x3,y
0,0,0,1,0
1,0,1,1,1
2,1,0,1,1
3,0,1,0,1
4,1,0,0,1
5,1,1,1,0
6,0,0,0,0


In [2]:
features = ['x1', 'x2', 'x3']
X = df[features]
y = df['y']

X

,x1,x2,x3
0,0,0,1
1,0,1,1
2,1,0,1
3,0,1,0
4,1,0,0
5,1,1,1
6,0,0,0


In [3]:
class Perceptron:
    def __init__(self, inputs=3, hiddens=4, outputs=1, niter=100):
        self.inputs = inputs
        self.hiddenNodes = hiddens
        self.outputNodes = outputs
        self.niter = niter
        
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoidPrime(self, x):
        return x * (1 - x)
    
    def predict_proba(self, X):
        """
        Feeds data from the input layer to the hidden layer, then to the output layer.
        """
        
        self.hidden_sum = np.dot(X, self.weights1) + self.bias1
        self.activated_hidden = self.__sigmoid(self.hidden_sum)
        
        self.output_sum = np.dot(self.activated_hidden, self.weights2) + self.bias2
        self.activated_output = self.__sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backprop(self, X, y, output):
        """
        Backward propogate the output through the network to train it
        """
        self.output_error = np.array(y) - output.T # Need to transpose or the result is a matrix
        self.output_delta = self.output_error * self.__sigmoidPrime(output.T)
        
        self.z2_error = self.output_delta.T.dot(self.weights2.T)
        self.z2_delta = self.z2_error * self.__sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta)
        self.bias1 += np.sum(self.z2_delta)
        self.weights2 += self.activated_hidden.T.dot(self.output_delta.T)
        self.bias2 += np.sum(self.output_delta)
        
        return self
        
    def fit(self, X, y):
        # Initial weights; putting these here so we get a new set every time 'fit' is called
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.bias1 = np.random.rand()
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        self.bias2 = np.random.rand()
        
        for _ in range(self.niter):
            output = self.predict_proba(X)
            self.backprop(X, y, output)
        
        return self
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.round(proba)

In [4]:
nn = Perceptron(niter=1000)
nn.fit(X, y)
y_pred = nn.predict(X)
y_pred

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [5]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

C:\Anaconda3\envs\Unit4_Sprint2\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\Unit4_Sprint2\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\Unit4_Sprint2\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\Unit4_Sprint2\lib\site-packages\tensorflow\python\framework\dtypes.

In [6]:
# input image dimensions
img_rows, img_cols = 28, 28

In [7]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [9]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [10]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [11]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [53]:
##### Your Code Here #####

nn = Perceptron(inputs=784, hiddens=15, niter=150)
nn.fit(x_train, y_train)
y_pred = nn.predict(x_test)

In [54]:
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [55]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

# Pretty good...but only because I got all zeroes

0.902

In [37]:
class Simple_Network:
    def __init__(self, inputs, hidden_layers, hidden_nodes, outputs=1, niter=1000):
        self.inputs = inputs
        self.hidden_layers = hidden_layers
        self.hiddenNodes = hidden_nodes
        self.outputNodes = outputs
        self.niter = niter
        
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoidPrime(self, x):
        return x * (1 - x)
    
    def predict_proba(self, X):
        """
        Feeds data from the input layer to the hidden layer(s), then to the output layer.
        """
        self.hidden_inout = X
        self.activated_hiddens = [X]
        for i in range(self.hidden_layers):
            self.hidden_sum = np.dot(self.hidden_inout, self.weights[i]) + self.biases[i]
            self.hidden_inout = self.__sigmoid(self.hidden_sum)
            self.activated_hiddens.append(self.hidden_inout)
        
        self.output_sum = np.dot(self.hidden_inout, self.weights[-1]) + self.biases[-1]
        self.activated_output = self.__sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backprop(self, X, y, output):
        """
        Backward propogate the output through the network to train it
        """
        self.output_error = np.array(y) - output.T # Need to transpose or the result is a matrix
        self.deltas = [self.output_error * self.__sigmoidPrime(output.T)]
        
        for i in range(self.hidden_layers):
                self.z2_error = self.deltas[i].T.dot(self.weights[-(i+1)].T)
                self.deltas.append((self.z2_error * self.__sigmoidPrime(self.activated_hiddens[-(i+1)])).T)
                self.weights[-(i+1)] += self.activated_hiddens[-(i+1)].T.dot(self.deltas[i].T)
                self.biases[-(i+1)] += np.sum(self.deltas[i])
       
        return self
        
    def fit(self, X, y):
        # Initial weights; putting these here so we get a new set every time 'fit' is called
        self.weights = []
        self.biases = []
        for i in range(self.hidden_layers + 1):
            self.biases.append(np.random.rand())
            if i == 0:
                self.weights.append(np.random.rand(self.inputs, self.hiddenNodes[i]))
            elif i == self.hidden_layers:
                self.weights.append(np.random.rand(self.hiddenNodes[i-1], self.outputNodes))
            else:
                self.weights.append(np.random.rand(self.hiddenNodes[i-1], self.hiddenNodes[i]))
        
        for _ in range(self.niter):
            output = self.predict_proba(X)
            self.backprop(X, y, output)
        
        return self
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.round(proba)

In [58]:
nn = Simple_Network(inputs=784, hidden_layers=2, hidden_nodes=[30, 15])
nn.fit(x_train, y_train)
y_pred = nn.predict(x_test)

In [59]:
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [60]:
accuracy_score(y_test, y_pred)

# Again, I just got all zeroes.

0.902

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?